# Multi-armed bandit problem

### Various algorithms that optimize the problem by employing explore/exploit strategies

**Namely**:

 * Random (for test purposes)
 * $\varepsilon$-Greedy
 * Decaying $\varepsilon$-Greedy (2 decay schemes)
 * Optimistic initial value
 * Upper confidence bound (UCB)
 * Thompson sampling (Gaussian, binary, and an experimental hybrid)

**Note:** these algorithms were developed for bound output [0,1] (or at least normal distribution with $\mu\in$~ (0-1] and $\sigma^2\approx$1) and I will demonsrate below where some of them underperform at certain conditions

**Note 2:** all demonstrations below are based on random data, and a therefore prone to stochasticity (if rerun using different seeds)

In [1]:
import numpy as np

from multi_armed import *

## Bandits

**aka the slot machines the player tries to choose from**

In [2]:
# bandit with a random output normally distributed
# Note: output is pregenerated so that the different algorithms can be compared using the same win/loss values
class Bandit():
    
    def __init__(self,size,E=0,s=1,variance_on_mean=False):
        # initial mean is chosen at random
        if (variance_on_mean):
            self.mean = np.random.normal(E,s)
        else:
            self.mean = np.random.normal(E,1)            
        # pre-generate all the data *+1 because counts start from 1 (see multi_armed.py)
        # always affected by variance
        self.data = np.random.normal(self.mean,s,size=size+1)
    
    def play(self,i):
        return self.data[i]

In [3]:
# bandit with a binary win/lose output
class BinaryBandit():
    
    def __init__(self,size,mean):
        self.mean = mean
        self.data = np.random.rand(size+1)
    
    def play(self,i):
        output = self.data[i]
        # return 0 or 1
        return int(output<self.mean)

**Generate datasets to test players on**

In [4]:
plays = 10000
no_of_bandits = 4

# numerical dataset
n_bandits = []
for i in range(no_of_bandits):
    # normal distribution (e=1, s=1)
    n_bandits.append(Bandit(plays,1))
    
# binary dataset
b_bandits = []
for i in range(no_of_bandits):
    b_bandits.append(BinaryBandit(plays,np.random.rand()))
    
# test player method
def test_player(player,bandits,plays):
    player.start(bandits)
    for i in range(plays):
        # choose bandit to play
        bandit = player.choose()
        # play bandit
        outcome = bandits[bandit].play(i)
        # update player
        player.update(bandit,outcome)
    return player.profit

## Players

### Numerical bandits

**Mean maximum**

In [5]:
sum = 0
best = n_bandits[np.argmax([b.mean for b in n_bandits])]
for i in range(plays):
    sum+=best.play(i)
int(sum)

19379

In [6]:
players = []

**Random player**

In [7]:
player = RandomPlayer()
players.append(player)
int(test_player(player,n_bandits,plays))

12763

**Epsilon greedy**

In [8]:
player = EpsilonGreedy(0.01)
players.append(player)
int(test_player(player,n_bandits,plays))

19018

In [9]:
player = EpsilonGreedy(0.2)
players.append(player)
int(test_player(player,n_bandits,plays))

18075

In [10]:
player = EpsilonGreedy(0.0001)
players.append(player)
int(test_player(player,n_bandits,plays))

16070

Note: value of epsilon needs to be experimented for best results
<br><br>
Otherwise, use:

**Dacaying epsilon greedy**

Epsilon value decays over time, assuming convergence

**Scheme 1:** Epsilon decays as b/N where N is the number of plays so far

In [11]:
player = DecayingEpsilonGreedy(b=1)
players.append(player)
int(test_player(player,n_bandits,plays))

16067

1/N is sometimes too fast, and the exploration is too short. User should provide b > number of bandits

In [12]:
player = DecayingEpsilonGreedy(b=10)
players.append(player)
int(test_player(player,n_bandits,plays))

19307

But not too big either:

In [13]:
player = DecayingEpsilonGreedy(b=1000)
players.append(player)
int(test_player(player,n_bandits,plays))

17188

**Scheme 2:** Epsilon decays by a fixed fraction (the decay rate)

In [14]:
player = DecayingEpsilonGreedy(decay=0.95)
players.append(player)
int(test_player(player,n_bandits,plays))

19366

Once again, performance depends on the rate chosen

In [15]:
player = DecayingEpsilonGreedy(decay=0.5)
players.append(player)
int(test_player(player,n_bandits,plays))

16068

In [16]:
player = DecayingEpsilonGreedy(decay=0.999)
players.append(player)
int(test_player(player,n_bandits,plays))

18747

**Optimistic initial value**

In [17]:
player = OptimsiticInitialValue(10)
players.append(player)
int(test_player(player,n_bandits,plays))

19356

Initial value must indeed be optimistic (higher than the best bandit's mean)

In [18]:
player = OptimsiticInitialValue(0.2)
players.append(player)
int(test_player(player,n_bandits,plays))

16068

Too high values will take longer to converge

In [19]:
player = OptimsiticInitialValue(1000)
players.append(player)
int(test_player(player,n_bandits,plays))

16890

**Upper confidence bound**

In [20]:
player = UCB()
players.append(player)
int(test_player(player,n_bandits,plays))

19306

Note: UCB can take a variance parameter, see below

**Thompson sampling (Gaussian)**

In [21]:
player = ThompsonSampling()
players.append(player)
int(test_player(player,n_bandits,plays))

19343

Note: TS can also take a variance parameter, see below

**Thompson sampling (Binary)**

In this context, 0 for any loss and 1 for any profit

In [22]:
player = BinaryThompsonSampling()
players.append(player)
int(test_player(player,n_bandits,plays))

19133

**Thompson sampling (Experimental hybrid)**

In [23]:
player = SelfRegulatingThompsonSampling()
players.append(player)
int(test_player(player,n_bandits,plays))

19267

### Binary bandits

**Mean maximum**

In [24]:
sum = 0
best = b_bandits[np.argmax([b.mean for b in b_bandits])]
for i in range(plays):
    sum+=best.play(i)
int(sum)

8809

**All players:**

In [25]:
for player in players:
    print(player)
    print('\t',int(test_player(player,b_bandits,plays)))

RandomPlayer
	 6764
EpsilonGreedy epsilon: 0.01
	 8580
EpsilonGreedy epsilon: 0.2
	 8389
EpsilonGreedy epsilon: 0.0001
	 7789
DecayingEpsilonGreedyb: 1
	 7791
DecayingEpsilonGreedyb: 10
	 8796
DecayingEpsilonGreedyb: 1000
	 8122
DecayingEpsilonGreedydecay: 0.95
	 7790
DecayingEpsilonGreedydecay: 0.5
	 7790
DecayingEpsilonGreedydecay: 0.999
	 7790
OptimisticInitialValue:10
	 8778
OptimisticInitialValue:0.2
	 8809
OptimisticInitialValue:1000
	 7385
UpperConfidenceBound
	 8663
GaussianThompsonSampling
	 8734
BinaryThompsonSampling
	 8794
SelfRegulatingThompsonSampling(Experimental)
	 8793


The general trend is similar to the numerical bandits

### Failure cases

Specific cases where some players fail will be explored, and possible solutions will be demonstrated.
<br>
Note: the experimental `SelfRegulatingThompsonSampling` has no hyper-parameters, and cannot be tweaked to succeed, but overall performs decently in all cases with no intervention

**High variance**

In [26]:
hv = []
for i in range(no_of_bandits):
    # note that the means are not affected by the variance
    # (this is important for the demonstration, because otherwis there is one bandit that is obviously
        # superior, and little sophistication is required to spot it)
    hv.append(Bandit(plays,1,20))

# subset of players to test
players = []
players.append(DecayingEpsilonGreedy(b=1))
players.append(DecayingEpsilonGreedy(decay=0.95))
players.append(OptimsiticInitialValue(10))
players.append(UCB())
players.append(ThompsonSampling())
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

In [26]:
for player in players:
    print(player)
    print('\t',int(test_player(player,hv,plays)))

DecayingEpsilonGreedyb: 1
	 23598
DecayingEpsilonGreedydecay: 0.95
	 8217
OptimisticInitialValue:10
	 16550
UpperConfidenceBound
	 -4786
GaussianThompsonSampling
	 16623
BinaryThompsonSampling
	 21749
SelfRegulatingThompsonSampling(Experimental)
	 20061


High variance breaks many of the players, because they set their heuristic assuming variance of 1
<br>
This can be corrected, by providing the players with estimated variance (or bounds taking the potential variance into account)

In [27]:
# corrected values
players = []
# high variance will require more time to converge, increase b
players.append(DecayingEpsilonGreedy(b=20))
# performance was ok
players.append(DecayingEpsilonGreedy(decay=0.8))
# high variance requires an overly optimistic value
players.append(OptimsiticInitialValue(50))
# provide estimate of variance
players.append(UCB(10))
# same
players.append(ThompsonSampling(10))
# performance was ok
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,hv,plays)))

DecayingEpsilonGreedyb: 20
	 24181
DecayingEpsilonGreedydecay: 0.8
	 24326
OptimisticInitialValue:50
	 24567
UpperConfidenceBound variance: 10
	 22022
GaussianThompsonSampling variance: 10
	 21836
BinaryThompsonSampling
	 20266
SelfRegulatingThompsonSampling(Experimental)
	 19238


**High bias** (all means are very positive)

In [28]:
hb = []
for i in range(no_of_bandits):
    # note that the variance affects the means in this case
    # If all means are high but the variance of the means is low, 
        # all bandits are nearly equal, and no strategy is needed)
    hb.append(Bandit(plays,20,5,True))

# subset of players to test
players = []
players.append(DecayingEpsilonGreedy(b=1))
players.append(DecayingEpsilonGreedy(decay=0.95))
players.append(OptimsiticInitialValue(10))
players.append(UCB())
players.append(ThompsonSampling())
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,hb,plays)))

DecayingEpsilonGreedyb: 1
	 319818
DecayingEpsilonGreedydecay: 0.95
	 258391
OptimisticInitialValue:10
	 177943
UpperConfidenceBound
	 177943
GaussianThompsonSampling
	 177943
BinaryThompsonSampling
	 263992
SelfRegulatingThompsonSampling(Experimental)
	 319287


If all bandits have high profit (but are quite different) many algorithms break, because they observe high outcome and assume this is unique and not universal, and no further optimization is performed

In [29]:
# corrected values
players = []
# performce is mostly affected by the variance itself rather than the bias
players.append(DecayingEpsilonGreedy(b=20))
# performance was ok
players.append(DecayingEpsilonGreedy(decay=0.65))
# initial value must by higher than the highest reasonable outcome
players.append(OptimsiticInitialValue(50))
# artificially high variance could overcome the issue, but the performance will still be hindered
players.append(UCB(20))
# same
players.append(ThompsonSampling(20))
# Binary TS cannot be fixed in this case - the data must be normalized
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,hb,plays)))

DecayingEpsilonGreedyb: 20
	 319210
DecayingEpsilonGreedydecay: 0.65
	 320241
OptimisticInitialValue:50
	 320173
UpperConfidenceBound variance: 20
	 318401
GaussianThompsonSampling variance: 20
	 319162
BinaryThompsonSampling
	 217103
SelfRegulatingThompsonSampling(Experimental)
	 319181


**Many iterations**

Whereas most algorithms tend to converge and perform better with more iterations, the nondecaying `EpsilonGreedy` will actually underperform after some time. <br><br>
Also, the `SelfRegulatingThompsonSampling` will underperform (see `multi_armed.py` for more details).
<br><br> In practice though, this doesn't seem to be very apparent

In [30]:
long = []
steps = 1000000

for i in range(no_of_bandits):
    # note that the variance affects the means in this case
    # If all means are high but the variance of the means is low, 
        # all bandits are nearly equal, and no strategy is needed)
    long.append(BinaryBandit(steps,np.random.rand()))

# subset of players to test
players = []
players.append(EpsilonGreedy(0.01))
players.append(DecayingEpsilonGreedy(decay=0.95))
players.append(ThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,long,steps)))

EpsilonGreedy epsilon: 0.01
	 915178
DecayingEpsilonGreedydecay: 0.95
	 917779
GaussianThompsonSampling
	 917610
SelfRegulatingThompsonSampling(Experimental)
	 917769


**Very small values**

In [31]:
small = []
for i in range(no_of_bandits):
    # note that the variance affects the means in this case
    # If all means are high but the variance of the means is low, 
        # all bandits are nearly equal, and no strategy is needed)
    small.append(Bandit(plays,0.001,0.001,True))

# subset of players to test
players = []
players.append(DecayingEpsilonGreedy(b=1))
players.append(DecayingEpsilonGreedy(decay=0.95))
players.append(OptimsiticInitialValue(10))
players.append(UCB())
players.append(ThompsonSampling())
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,small,plays)))

DecayingEpsilonGreedyb: 1
	 24
DecayingEpsilonGreedydecay: 0.95
	 24
OptimisticInitialValue:10
	 10
UpperConfidenceBound
	 8
GaussianThompsonSampling
	 8
BinaryThompsonSampling
	 24
SelfRegulatingThompsonSampling(Experimental)
	 24


The small values and small variance take too long to converge for some algorithms

In [32]:
# corrected values
players = []
# performance was ok
players.append(DecayingEpsilonGreedy(b=1))
# performance was ok
players.append(DecayingEpsilonGreedy(decay=0.95))
# optimistic value needs to be smaller
players.append(OptimsiticInitialValue(0.1))
# provide small variance
players.append(UCB(0.01))
# same
players.append(ThompsonSampling(0.01))
# performance was ok
players.append(BinaryThompsonSampling())
# performance was ok
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,small,plays)))

DecayingEpsilonGreedyb: 1
	 24
DecayingEpsilonGreedydecay: 0.95
	 24
OptimisticInitialValue:0.1
	 24
UpperConfidenceBound variance: 0.01
	 22
GaussianThompsonSampling variance: 0.01
	 23
BinaryThompsonSampling
	 24
SelfRegulatingThompsonSampling(Experimental)
	 24


**Many bandits** (or few iterations compared to the number of bandits)

In [33]:
many = []
many_bandits = 100

for i in range(many_bandits):
    # note that the variance affects the means in this case
    # If all means are high but the variance of the means is low, 
        # all bandits are nearly equal, and no strategy is needed)
    many.append(Bandit(plays,1))

# subset of players to test
players = []
players.append(DecayingEpsilonGreedy(b=1))
players.append(DecayingEpsilonGreedy(decay=0.95))
players.append(OptimsiticInitialValue(10))
players.append(UCB())
players.append(ThompsonSampling())
players.append(BinaryThompsonSampling())
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,many,plays)))

DecayingEpsilonGreedyb: 1
	 14597
DecayingEpsilonGreedydecay: 0.95
	 24949
OptimisticInitialValue:10
	 36865
UpperConfidenceBound
	 37348
GaussianThompsonSampling
	 37260
BinaryThompsonSampling
	 32318
SelfRegulatingThompsonSampling(Experimental)
	 34892


The `DecayingEpsilonGreedy`s decay too fast to explore that many bandits. BinaryThompson probably does not converge.

In [34]:
# corrected values
players = []
# decrease decay
players.append(DecayingEpsilonGreedy(b=100))
# decrease decay
players.append(DecayingEpsilonGreedy(decay=0.9))
players.append(ThompsonSampling())
# Nothing can be done, run more iterations if possible
players.append(BinaryThompsonSampling())
# performance was ok
players.append(SelfRegulatingThompsonSampling())

for player in players:
    print(player)
    print('\t',int(test_player(player,many,plays)))

DecayingEpsilonGreedyb: 100
	 36687
DecayingEpsilonGreedydecay: 0.9
	 34706
GaussianThompsonSampling
	 37681
BinaryThompsonSampling
	 32733
SelfRegulatingThompsonSampling(Experimental)
	 34291


<br>

### Conclusion

**Each algorithm has cases where it underperforms, choice should be case-dependent**

**As a rule of thumb though, `DecayingEpsilonGreedy` appear to perform universally well**
The many bandits is a special case that should be known (as opposed to low/high values or variance, that is not accessible in a real scenario) and so the `DecayingEpsilonGreedy` is probably the safest choise